![Logo Uni Köln](https://raw.githubusercontent.com/jmelsbach/ai-im/main/img/uni-logo.png)

# Exercise 01 Notebook -  Image Classification with a Neural Network

In this exercise you will implement a neural network from scratch using pytorch. We train the model on the fashion MNIST dataset. The Goal is to find a good architecture and hyperparameters to achieve the highest test score possible!

Here are the exact steps we need to take:
1. Download the dataset and create a dataset for training, validation and test data.
2. Explore and understand the dataset.
3. Create DataLoader for training, validation and test set.
4. Create a Neural Network Architecture that fits the problem.
5. Set the hyperparameters and choose a suitable loss function.
6. Create a training loop
7. Train the model

We'll use the Fashion-MNIST data set during this exercise. The Fashion-MNIST dataset consists of 60,000 training examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 different classes:

    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot"

![Fashion MNIST Long](https://raw.githubusercontent.com/jmelsbach/ai-im/main/img/fashion-mnist_long.png)

In [ ]:
# Imports
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Library for printing out progress bars
# This can be useful in the training loop
# The use is optional
from tqdm.notebook import tqdm

## 1. Download the dataset and create a dataset for training, validation and test data.
You can learn how to download the training dataset [here](https://pytorch.org/vision/stable/datasets.html#fashion-mnist). Use the documentation to download the training and test set of the Fashion-MNIST Dataset. Out of the box the images of the dataset have the PIL format but we need them as `torch.Tensors` to feed them in our neural network later on. <br>
**Hint**: Use a transform to convert the PIL to the Tensor format. You can learn about transformations [here](https://pytorch.org/vision/stable/transforms.html).

In [ ]:
train_data = torchvision.datasets.FashionMNIST('data', download=True, transform=transforms.ToTensor())
test_data = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transforms.ToTensor())

## 2. Explore and understand the dataset.
If you successfully created the dataset objects, try to explore the data.
Answer the following questions:
* How many training examples do we have?
* How many test examples do we have?
* What type of datastructure is each datapoint?
* Get the shape of the a training image. What does each dimensions mean?
    * You notice that the shape is a little bit awkward. We'll deal with this later in the `forward()` method of our neural network
* Do we need to normalize the data?
* Plot a random image and the corresponding label from the dataset with the help of the `matplotlib`library.

In [ ]:
# Number of training and test examples
len(train_data), len(test_data)

In [ ]:
# get type of training example
type(train_data[0])

In [ ]:
type(train_data[0][0])

In [ ]:
# image
train_data[0][0].shape

In [ ]:
# label
train_data[0][1]

In [ ]:
# Do we need to normalize?
train_data[0][0].max(), train_data[0][0].min()

In [ ]:
# Plot random image
import matplotlib.pyplot as plt

In [ ]:
random_index = torch.randint(len(train_data), size=(1,)).item()
# or
random_index = torch.randint(len(train_data), size=(1,))[0]

In [ ]:
img = train_data[random_index][0].numpy()
label = train_data[random_index][1]

In [ ]:
print(label)
plt.imshow(img.squeeze())

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3,3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## 3. Create DataLoader for training, validation and test set.

We do not have a validation set, yet. Split the `train_data` with the help of the `random_split`. Look at the documentation of the random_split function [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split). Split the data in a 80:20 train/val ratio.




In [ ]:
from torch.utils.data import random_split

In [ ]:
train, val = random_split(train_data, [48_000, 12_000],)

* Create a `torch.utils.data.DataLoader` for train, val and test data.
* Use a batch size of 32.
* Don't forget to shuffle the data!

In [ ]:
from torch.utils.data import DataLoader
train_dl = DataLoader(train, batch_size=32, shuffle = True)
val_dl   = DataLoader(val, batch_size=32, shuffle=False)
test_dl  = DataLoader(test_data, batch_size=32, shuffle=False)

## 4. Create a Neural Network Architecture that fits the problem.

Create a Neural Network with two hidden layer of size 20 each. Choose the correct input and output size suitable for the problem.


In [ ]:
class NeuralNetwork(nn.Module):

    def __init__(self,):
        # this line always has to be at the beginning
        # of a new Module
        super().__init__()
        self.fc1 = nn.Linear(784, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20,10)


    def forward(self, X):
        # We need to bring the data into a format our
        # neural network can handle. Try to understand
        X = X.reshape(X.size(0), -1)
        # Pass the input through your layer and add sigmoid activation function
        X = self.fc1(X)
        X = torch.sigmoid(X)
        X = self.fc2(X)
        X = torch.sigmoid(X)
        X = self.fc3(X)

        return X

Instantiate the `NeuralNetwork`

In [ ]:
model = NeuralNetwork()

## 5. Set the hyperparameters and choose a suitable loss function.

Instantiate an optimizer and a loss function. Use stochastic gradient descent as your optimizer and pick a suitable loss function for the data. You can look up how to create an optimizer [here](https://pytorch.org/docs/stable/optim.html).

In [ ]:
epochs = 10
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

## 6. Create a training loop

The trainig loop should receive the `net`, `train_dl`, `val_dl`, `epochs`, `optimizer`, and `loss_func`.
Print out the average loss and the accuarcy on both the train **and** valiadtion data.

In [ ]:
# To get a better idea of how well your model performs
# you should implement an accuracy function that is
# called after each epoch of your training loop
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
def train(model, train_dl, val_dl , epochs, optimizer, loss_func):
    print("epoch | train loss | train acc | val loss | val acc")

    for epoch in range(epochs):
      # set model in training state
      model.train()
      total_acc = 0
      total_loss = 0

      for xb, yb in train_dl:

        # make the prediction
        pred = model(xb)
        # calc the loss
        loss = loss_func(pred, yb)
        # zero the all gradients
        # calc gradients
        loss.backward()
        # update weights
        optimizer.step()

        optimizer.zero_grad()



        total_loss += loss
        total_acc += accuracy(pred, yb)

      total_loss /= len(train_dl)
      total_acc /= len(train_dl)

      total_acc_val = 0
      total_loss_val = 0
      model.eval()
      with torch.no_grad():
        for xb_val, yb_val in val_dl:
          pred_val = model(xb_val)
          loss_val = loss_func(pred_val, yb_val)
          total_loss_val += loss_val
          total_acc_val += accuracy(pred_val, yb_val)


      total_acc_val /= len(val_dl)
      total_loss_val /= len(val_dl)
      print("---------------------------------------------------")
      print(f"  {epoch}   |    {total_loss.item():.4f}  |  {total_acc.item():.4f}   |  {total_loss_val.item():.4f}  |   {total_acc_val.item():.4f} ")

## 7. Train the model

In [ ]:
# Execute the train function and train the model.
train(model, train_dl, val_dl, epochs, optimizer, loss_func)

# 8. Check your test performance

In [ ]:
test_acc = 0
with torch.no_grad():
    # Perform a prediction on the test set
    for xb_test, yb_test in test_dl:
        model.eval()
        pred_test = model(xb_test)  # Forward pass
        acc = accuracy(pred_test, yb_test)
        test_acc += acc

    test_acc /= len(test_dl)

print(f"Test Accuracy is: {test_acc:.2f}")